In [11]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from requests import get
import requests
import re
import urllib
import os
import glob
from tqdm import tqdm
from datetime import datetime
import time
pd.set_option('display.max_colwidth', None)
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_funder2(pid):
    
    start_time = time.time()
    
    stopwords = ['Note','',' ','Funding agencies:','Funding Agencies:','Sponsored by',': ','Forskningsfinansiär', 'Funding Agency:']
    stopwords = '|'.join(stopwords)
    regex = re.compile(r'\b('+stopwords+r')\b', flags=re.IGNORECASE) 
    
    fund_dict = {}
        
    for k in pid:
        
        url = 'https://oru.diva-portal.org/smash/record.jsf?dswid=-2429&pid=diva2%3A'+str(k)+'&c=1&searchType=SIMPLE&language=en&query='+str(k)+'&af=%5B%5D&aq=%5B%5B%5D%5D&aq2=%5B%5B%5D%5D&aqe=%5B%5D&noOfRows=50&sortOrder=author_sort_asc&sortOrder2=title_sort_asc&onlyFullText=false&sf=all'
        page = get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        fund_name=[]
        key = k
        
        t = ''
        for i in range(0, len(soup.find_all('h5'))):
            if(soup.find_all('h5')[i].text=='Note'):
                t = soup.find_all('h5')[i].next_sibling
        data = []
        if(t != ''):
            while(t.string != None):
                if((t.string not in stopwords) & (len(t.string) < 300) & (t.string != None) & (t.string != '\n')):
                    data.append(t.string)
                t = t.next_sibling
        
            fund_dict[str(key)] = data
    
    df = pd.DataFrame(list(fund_dict.values()), index=fund_dict.keys()).reset_index().rename(columns={'index':'pid'})
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return df

In [12]:
def getHits():
    # Set up headless browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    
    try:
        driver = webdriver.Chrome(options=options)
        url = 'https://oru.diva-portal.org/smash/resultList.jsf?dswid=-5296&searchType=SUBJECT&language=en&onlyFullText=false&af=%5B%5D&aq=%5B%5B%7B%22categoryId%22%3A%2211649%22%7D%5D%5D'
        
        driver.get(url)
        
        # Wait for the pagination element to be visible
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "paginInformation"))
        )
        
        # Find the pagination information
        pagination_info = driver.find_element(By.CLASS_NAME, "paginInformation").text
        
        # Extract the hit count
        hits_match = re.search(r'of (\d+)', pagination_info)
        result = hits_match.group(1) if hits_match else pagination_info
        
        driver.quit()
        return result
        
    except Exception as e:
        print(f"Error with Selenium: {e}")
        try:
            driver.quit()
        except:
            pass
        return None

In [13]:
print(getHits())


22654


In [14]:
def download_data(file_url, filename):
    directory ="./raw_data/"
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    linkToFile = file_url
    localDestination = "./raw_data/"+filename
    resultFilePath, responseHeaders = urllib.request.urlretrieve(linkToFile, localDestination)

In [15]:
%%time
hits = getHits()
#diva_url = "https://oru.diva-portal.org/smash/export.jsf?format=csv&addFilename=true&aq=[[{%22categoryId%22:%2211649%22}]]&aqe=[]&aq2=[[]]&onlyFullText=false&noOfRows="+str(hits)+"&sortOrder=title_sort_asc&sortOrder2=title_sort_asc&csvType=publication&fl="

auth_df_cols = ['PID','AuthorityPid','DOI','FirstName','ISI','ISRN','LastName','LocalId','NBN','ORCID',
                'OrganisationId','UncontrolledOrganisation','Position','PMID','ResearchGroup','Role',
                'ScopusId']

pubs_df_cols = ['PID','Abstract','ArchiveNumber','ArticleId','Name','City','Conference','ContentType',
                'CreatedDate','Credits','DOI','DefenceDate','DefenceLanguage','DefencePlace',
                'Edition','EndPage','Examiners','FullTextLink','Funder','HostPublication',
                'HostPublicationAll','ISBN','ISBN_ELECTRONIC','ISBN_PRINT','ISI','ISRN',
                'Issue','Journal','JournalEISSN','JournalISSN','Keywords','Language','LastUpdated',
                'Urls','LocalId','NBN','Categories','Notes','NumberOfAuthors','PersonCount',
                'Opponents','Pages','PartOfThesis','Patent','Programme','Projects','PublicationDate',
                'PublicationStatus','PublicationType','Publisher','PMID','ResearchSubjects','ScopusId',
                'SeriesTitle','SeriesEISSN','SeriesISSN','StartPage','Subject','Supervisors',
                'ThesisLevel','Title','TypeOfContent','Uppsok','Volume','Year']


onepub_url = '''https://oru.diva-portal.org/smash/export.jsf?format=csv&addFilename=true&aq=[[{"categoryId":"11649"},{"mediaTypeCode":"\"text\""}]]&aqe=[]&aq2=[[{"language":"eng"},{"publicationTypeCode":["bookReview","dissertation","review","comprehensiveDoctoralThesis","article","monographDoctoralThesis","comprehensiveLicentiateThesis","book","monographLicentiateThesis","chapter","manuscript","conferencePaper","conferenceProceedings"]}]]&onlyFullText=false&noOfRows='''+str(hits)+'''&sortOrder=title_sort_asc&sortOrder2=title_sort_asc&csvType=publication&fl=PID,Abstract,ArchiveNumber,ArticleId,Name,City,Conference,ContentType,CreatedDate,Credits,DOI,DefenceDate,DefenceLanguage,DefencePlace,Edition,EndPage,Examiners,FullTextLink,Funder,HostPublication,HostPublicationAll,ISBN,ISBN_ELECTRONIC,ISBN_PRINT,ISI,ISRN,Issue,Journal,JournalEISSN,JournalISSN,Keywords,Language,LastUpdated,Urls,LocalId,NBN,Categories,Notes,NumberOfAuthors,PersonCount,Opponents,Pages,PartOfThesis,Patent,Programme,Projects,PublicationDate,PublicationStatus,PublicationType,Publisher,PMID,ResearchSubjects,ScopusId,SeriesTitle,SeriesEISSN,SeriesISSN,StartPage,Subject,Supervisors,ThesisLevel,Title,TypeOfContent,Uppsok,Volume,Year'''
authors_url = '''https://oru.diva-portal.org/smash/export.jsf?format=csv&addFilename=true&aq=[[{"categoryId":"11649"},{"mediaTypeCode":"\"text\""}]]&aqe=[]&aq2=[[{"language":"eng"},{"publicationTypeCode":["bookReview","dissertation","review","comprehensiveDoctoralThesis","article","monographDoctoralThesis","comprehensiveLicentiateThesis","book","monographLicentiateThesis","chapter","manuscript","conferencePaper","conferenceProceedings"]}]]&onlyFullText=false&noOfRows='''+str(hits)+'''&sortOrder=title_sort_asc&sortOrder2=title_sort_asc&csvType=person&fl=PID,AuthorityPid,DOI,FirstName,ISI,ISRN,LastName,LocalId,NBN,ORCID,OrganisationId,UncontrolledOrganisation,Position,PMID,ResearchGroup,Role,ScopusId'''


if os.path.exists("./raw_data/"+"publications_"+str(hits)+".csv") == False:
    download_data(onepub_url, "publications_"+str(hits)+".csv")
    pubs_df = pd.read_csv("./raw_data/"+"publications_"+str(hits)+".csv", names=pubs_df_cols, header=0)
else:
    pubs_df = pd.read_csv("./raw_data/"+"publications_"+str(hits)+".csv", names=pubs_df_cols, header=0)

CPU times: user 806 ms, sys: 434 ms, total: 1.24 s
Wall time: 2min 16s


In [20]:
if os.path.exists("./raw_data/"+"authors_"+str(hits)+".csv") == False:
    download_data(authors_url, "authors_"+str(hits)+".csv")
    authors_df = pd.read_csv("./raw_data/"+"authors_"+str(hits)+".csv", names=auth_df_cols, header=0, usecols=[i for i in range(0,17)])
else:
    authors_df = pd.read_csv("./raw_data/"+"authors_"+str(hits)+".csv", names=auth_df_cols, header=0, usecols=[i for i in range(0,17)])

In [21]:
authors_df.head()

,PID,AuthorityPid,DOI,FirstName,ISI,ISRN,LastName,LocalId,NBN,ORCID,OrganisationId,UncontrolledOrganisation,Position,PMID,ResearchGroup,Role,ScopusId
0,1595685,NaN,10.1007/978-3-642-40683-6_1,Martin,NaN,NaN,Theobald,NaN,urn:nbn:se:oru:diva-94458,NaN,NaN,"Universiteit Antwerpen, Antwerp, Belgium",1,NaN,NaN,aut,2-s2.0-84883289559
1,1595685,authority-person:77584,10.1007/978-3-642-40683-6_1,Luc,NaN,NaN,De Raedt,lcdt,urn:nbn:se:oru:diva-94458,0000-0002-6860-6303,NaN,"Katholieke Universiteit Leuven, Heverlee, Belgium",2,NaN,NaN,aut,2-s2.0-84883289559
2,1595685,NaN,10.1007/978-3-642-40683-6_1,Maximilian,NaN,NaN,Dylla,NaN,urn:nbn:se:oru:diva-94458,NaN,NaN,"Max Planck Institut Informatik, Saarbrücken, Germany",3,NaN,NaN,aut,2-s2.0-84883289559
3,1595685,NaN,10.1007/978-3-642-40683-6_1,Angelika,NaN,NaN,Kimmig,NaN,urn:nbn:se:oru:diva-94458,NaN,NaN,"Katholieke Universiteit Leuven, Heverlee, Belgium",4,NaN,NaN,aut,2-s2.0-84883289559
4,1595685,NaN,10.1007/978-3-642-40683-6_1,Iris,NaN,NaN,Miliaraki,NaN,urn:nbn:se:oru:diva-94458,NaN,NaN,"Max Planck Institut Informatik, Saarbrücken, Germany",5,NaN,NaN,aut,2-s2.0-84883289559


In [22]:
authors_df.shape

(111675, 17)

In [23]:
pubs_df.shape

(22649, 65)

In [35]:
pubs_df[~pubs_df['Abstract'].isnull()].shape

(17696, 65)

In [58]:
abstracts_df = pubs_df[~pubs_df['Abstract'].isnull()][['PID','Abstract','PMID']].copy()

In [59]:
abstracts_df.shape 

(17696, 3)

In [60]:
abstracts_df.to_csv('./raw_data/abstracts.csv', index=False)

In [62]:
abstracts_df[~abstracts_df['PMID'].isnull()].shape

(8626, 3)

# entities recognition

In [79]:
def query_plain(text, url="http://bern2.korea.ac.kr/plain", max_retries=3, retry_delay=2):
    """
    Query the BERN2 biomedical entity recognition service with retry logic
    
    Args:
        text (str): Text to annotate
        url (str): BERN2 API endpoint URL (using HTTPS)
        max_retries (int): Maximum number of retry attempts
        retry_delay (int): Delay between retries in seconds
        
    Returns:
        dict or str: JSON response or error message
    """
    for attempt in range(max_retries):
        try:
            response = requests.post(
                url, 
                json={'text': text},
                timeout=30,
                headers={'Content-Type': 'application/json'}
            )
            response.raise_for_status()  # Raise exception for 4XX/5XX responses
            return response.json()
        
        except requests.exceptions.ConnectionError as e:
            if attempt < max_retries - 1:
                print(f"Connection error occurred. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff
            else:
                return f"Connection error after {max_retries} attempts: {str(e)}"
                
        except requests.exceptions.RequestException as e:
            return f"Request error: {str(e)}"


In [80]:
#abstracts_df.loc[1][1]
query_plain(abstracts_df.loc[1][1])

{'annotations': [{'id': ['NCBITaxon:9606'],
   'is_neural_normalized': False,
   'mention': 'patients',
   'obj': 'species',
   'prob': 0.9807732105255127,
   'span': {'begin': 122, 'end': 130}},
  {'id': ['NCBITaxon:9606'],
   'is_neural_normalized': False,
   'mention': 'patients',
   'obj': 'species',
   'prob': 0.9915779829025269,
   'span': {'begin': 240, 'end': 248}},
  {'id': ['NCBITaxon:9606'],
   'is_neural_normalized': False,
   'mention': 'patients',
   'obj': 'species',
   'prob': 0.9891461133956909,
   'span': {'begin': 277, 'end': 285}},
  {'id': ['NCBITaxon:9606'],
   'is_neural_normalized': False,
   'mention': 'patients',
   'obj': 'species',
   'prob': 0.9875727891921997,
   'span': {'begin': 357, 'end': 365}},
  {'id': ['CUI-less'],
   'is_neural_normalized': False,
   'mention': 'HRA',
   'obj': 'DNA',
   'prob': 0.5451631546020508,
   'span': {'begin': 494, 'end': 497}},
  {'id': ['NCBITaxon:9606'],
   'is_neural_normalized': False,
   'mention': 'patients',
   'ob

In [65]:
abstracts_df.loc[1][1]

'Background: We analysed the long-term revision rate, clinical outcomes and metal ion concentrations in blood over time in patients who had undergone metal-on-metal Articular Surface Replacement (ASR) hip arthroplasty.Methods: A total of 38 patients (43 hips) were included: 24 patients (28 hips) underwent large-head total hip arthroplasty (XL THA), and 14 patients (15 hips) underwent hip resurfacing arthroplasty (HRA). The median follow-up time was 11 (range 7-12) years.Results: None of 15 HRA implants were revised. Nine of 28 XL THA implants (32%) in 8 patients were revised. The Co ion levels significantly increased in the XL THA group (p=0.009) over a median time period of 84 (25-97) months.Conclusion: The levels of Co ions in blood were higher in the patients who had undergone XL THA and increased significantly over time.'